In [1]:
import requests
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id ='f22713124fb04e97a26f46a51547744f',
                                                          client_secret='75c001d5dfc54f6cb6e44f7a54cdabc0'))
                     

In [3]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    while results['next']:
        #next page:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

## Testing first:

In [4]:
all_tracks = get_playlist_tracks('spotify', '4rnleEAOdmFAbRcNCgZMpY')

In [5]:
dic_id_song_artist = [{track['track']['id'] : [track['track']['name'] ,artist['name']]} for track in all_tracks[0:100] for artist in track['track']['artists']]

In [6]:
result = sp.audio_features('7dS5EaCoMnN7DzlpT6aRn2')[0]

In [7]:
audio_keys = list(result.keys())

In [8]:
#audio_keys

In [9]:
df_header = ['track_id','name','artist']
df_header.extend(audio_keys[0:11])

In [10]:
print(df_header)

['track_id', 'name', 'artist', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']


In [11]:
pd.DataFrame([{audio_keys[i] : result[audio_keys[i]] for i in range(11)}])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.566,0.664,4,-5.303,0,0.0464,0.634,0,0.116,0.437,128.945


In [12]:
#[{audio_keys[i] : result[audio_keys[i]] for i in range(11)}]

In [13]:
{audio_keys[i] : result[audio_keys[i]] for i in range(11)}

{'danceability': 0.566,
 'energy': 0.664,
 'key': 4,
 'loudness': -5.303,
 'mode': 0,
 'speechiness': 0.0464,
 'acousticness': 0.634,
 'instrumentalness': 0,
 'liveness': 0.116,
 'valence': 0.437,
 'tempo': 128.945}

## Creating a loop to extract audio features from a playlist:

In [14]:
all_tracks = get_playlist_tracks('spotify', '4rnleEAOdmFAbRcNCgZMpY')

In [15]:
all_tracks[1]['track']['artists'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2KsP6tYLJlTBvSUxnwlVWa'},
 'href': 'https://api.spotify.com/v1/artists/2KsP6tYLJlTBvSUxnwlVWa',
 'id': '2KsP6tYLJlTBvSUxnwlVWa',
 'name': 'Mike Posner',
 'type': 'artist',
 'uri': 'spotify:artist:2KsP6tYLJlTBvSUxnwlVWa'}

## Only getting first artist mentioned in a song

In [16]:
#df_header
#audio_keys
#dic_id_song_artist = [{track['track']['id'] : [track['track']['name'] ,artist['name']]} for track in all_tracks[0:100] for artist in track['track']['artists']]
dic_id_song_artist = [{track['track']['id'] : [track['track']['name'] ,track['track']['artists'][0]['name']]} for track in all_tracks[0:100]]

In [17]:
#len(dic_id_song_artist)

In [18]:
# dic_id_song_artist

In [19]:
df_song = pd.DataFrame(columns = df_header)

In [30]:
# df_song

## Testing appending rows to DF

In [27]:
# dic_key = list(dic_id_song_artist[0].keys())[0]
# result = sp.audio_features(dic_key)[0]
# dic_new_row = {df_header[0] : dic_key, 
#                 df_header[1] : dic_id_song_artist[0][dic_key][0],
#                 df_header[2] : dic_id_song_artist[0][dic_key][1]}
# dic_new_row.update({audio_keys[i] : result[audio_keys[i]] for i in range(11)})
# #print(dic_new_row)
# df_song.append(dic_new_row,ignore_index=True) 

{'track_id': '7dS5EaCoMnN7DzlpT6aRn2', 'name': 'Take Me To Church', 'artist': 'Hozier', 'danceability': 0.566, 'energy': 0.664, 'key': 4, 'loudness': -5.303, 'mode': 0, 'speechiness': 0.0464, 'acousticness': 0.634, 'instrumentalness': 0, 'liveness': 0.116, 'valence': 0.437, 'tempo': 128.945}


,track_id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,7dS5EaCoMnN7DzlpT6aRn2,Take Me To Church,Hozier,0.566,0.664,4,-5.303,0,0.0464,0.634,0,0.116,0.437,128.945


In [22]:
dic_new_row

{'track_id': '7dS5EaCoMnN7DzlpT6aRn2',
 'name': 'Take Me To Church',
 'artist': 'Hozier',
 'danceability': 0.566,
 'energy': 0.664,
 'key': 4,
 'loudness': -5.303,
 'mode': 0,
 'speechiness': 0.0464,
 'acousticness': 0.634,
 'instrumentalness': 0,
 'liveness': 0.116,
 'valence': 0.437,
 'tempo': 128.945}

In [23]:
# dic_id_song_artist

## Final loop for a specific Playlist:

In [28]:
for loop in range(len(dic_id_song_artist)):
    song_id = list(dic_id_song_artist[loop].keys())[0]
    result = sp.audio_features(song_id)[0]
    dic_new_row = {df_header[0] : song_id, 
                    df_header[1] : dic_id_song_artist[loop][song_id][0],
                    df_header[2] : dic_id_song_artist[loop][song_id][1]}
    dic_new_row.update({audio_keys[i] : result[audio_keys[i]] for i in range(11)})
    df_song = df_song.append(dic_new_row,ignore_index=True)

In [29]:
df_song

,track_id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,7dS5EaCoMnN7DzlpT6aRn2,Take Me To Church,Hozier,0.566,0.664,4,-5.303,0,0.0464,0.634000,0,0.1160,0.437,128.945
1,2V4bv1fNWfTcyRJKmej6Sj,Cooler Than Me - Single Mix,Mike Posner,0.768,0.820,7,-4.630,0,0.0475,0.179000,0,0.7040,0.625,129.969
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",0.558,0.559,6,-9.222,1,0.0959,0.371000,7.49e-06,0.1090,0.620,78.558
3,3gbBpTdY8lnQwqxNCcf795,Pompeii,Bastille,0.679,0.715,9,-6.383,1,0.0407,0.075500,0,0.2710,0.571,127.435
4,3ZFTkvIE7kyPt6Nu3PEa7V,Hips Don't Lie (feat. Wyclef Jean),Shakira,0.778,0.824,10,-5.892,0,0.0712,0.284000,0,0.4050,0.756,100.024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4JehYebiI9JE8sR8MisGVb,Halo,Beyoncé,0.508,0.720,11,-5.908,0,0.0628,0.272000,0,0.0563,0.472,79.983
96,7Lf7oSEVdzZqTA0kEDSlS5,Cry Me a River,Justin Timberlake,0.624,0.654,8,-6.582,0,0.1830,0.577000,0,0.1040,0.564,73.898
97,1FMm3wG5WOhi7js35KR7Ea,"You're Gonna Go Far, Kid",The Offspring,0.550,0.918,9,-3.159,0,0.0682,0.004660,0,0.2060,0.563,126.083
98,6qYGUxPjQt5PJtWdiNppZx,So What,P!nk,0.535,0.886,11,-3.106,1,0.0450,0.000304,2.26e-06,0.2570,0.445,126.012


## Transforming into a function:

In [44]:
def get_track_features_from_playlist(username,playlist_id):
    #get all tracks from playlist:
    all_tracks = get_playlist_tracks(username, playlist_id)
    #extract header for audio features:
    audio_keys = list(sp.audio_features(all_tracks[0]['track']['id'])[0].keys())[0:11]
    #create header for df:
    df_header = ['track_id','name','artist']
    df_header.extend(audio_keys)
    #create empty df with header names:
    df_song = pd.DataFrame(columns = df_header)
    #create dictionary as <song_id> : <songname>, <artist_name>
    dic_id_song_artist = [{track['track']['id'] : [track['track']['name'] ,track['track']['artists'][0]['name']]} for track in all_tracks]
    #loop through dictionary, for each song, get audio features and append to dataframe:
    for loop in range(len(dic_id_song_artist)):
        #get song id:
        song_id = list(dic_id_song_artist[loop].keys())[0]
        #get audio features:
        result = sp.audio_features(song_id)[0]
        #create temp dictionary in order to append to df:
        dic_new_row = {df_header[0] : song_id, 
                        df_header[1] : dic_id_song_artist[loop][song_id][0],
                        df_header[2] : dic_id_song_artist[loop][song_id][1]}
        dic_new_row.update({audio_keys[i] : result[audio_keys[i]] for i in range(len(audio_keys))})
        #append new row to dataframe (songid,songanme,artist,audio_features...)
        df_song = df_song.append(dic_new_row,ignore_index=True)
    return df_song

In [59]:
test = get_track_features_from_playlist('spotify','4rnleEAOdmFAbRcNCgZMpY')

In [62]:
test.shape

(5198, 14)

In [60]:
test.head()

,track_id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,7dS5EaCoMnN7DzlpT6aRn2,Take Me To Church,Hozier,0.566,0.664,4,-5.303,0,0.0464,0.6340,0,0.116,0.437,128.945
1,2V4bv1fNWfTcyRJKmej6Sj,Cooler Than Me - Single Mix,Mike Posner,0.768,0.820,7,-4.630,0,0.0475,0.1790,0,0.704,0.625,129.969
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",0.558,0.559,6,-9.222,1,0.0959,0.3710,7.49e-06,0.109,0.620,78.558
3,3gbBpTdY8lnQwqxNCcf795,Pompeii,Bastille,0.679,0.715,9,-6.383,1,0.0407,0.0755,0,0.271,0.571,127.435
4,3ZFTkvIE7kyPt6Nu3PEa7V,Hips Don't Lie (feat. Wyclef Jean),Shakira,0.778,0.824,10,-5.892,0,0.0712,0.2840,0,0.405,0.756,100.024


## Looping through multiple Playlist: